In [3]:
import math
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mysql.connector


mysql code is the next code chunk. 

In [15]:

config = {
    'user': 'root',
    'password': 'JJY#91296517',
    'host': '127.0.0.1',
    'database': 'database',
}

# Establish a connection
cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM combined_reviews"

# Use pandas to load sql query into a DataFrame
reviews = pd.read_sql(query, con=cnx)

print(reviews.head())  # Print the first few rows of the DataFrame

# Don't forget to close the connection when done
cnx.close()

                                        replyContent  score  \
0  Hi Hafidz Melek! Thank you for your 5-star rat...      5   
1  We're sorry to hear that you feel this way, Li...      1   
2  Hi Kh Lim. We're sorry to hear this. Please co...      1   
3  Hi Shaun. We're sorry to hear this and we woul...      1   
4                                               None      5   

                                             content  thumbsUpCount  
0  I don't get the negative reviews here. I insta...              0  
1  Not sure if i am already registered for the sa...              0  
2  developer is nutshell brain i think proof of a...              0  
3  i cant eve open the app now and purchase thing...              0  
4                                  Gif me mony plsse              0  


C:\Users\AngJi\AppData\Local\Temp\ipykernel_26512\920605474.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews = pd.read_sql(query, con=cnx)


In [16]:
# reviews = pd.read_csv('combined_reviews.csv', index_col=0)
# reviews.head()
reviews = reviews.drop(columns=['score', 'thumbsUpCount'])
reviews = reviews.dropna()
inst = 'Below is a review for a banking app. Write a response that appropriately replies to the review from the perspective of the bank.'
# text = []
# for row in range(len(reviews)):
#     text.append(f'{inst} ### Review: {reviews.content[row]} ### Reply: {reviews.replyContent[row]}')
# reviews['text'] = text
text = []
for index, row in reviews.iterrows():
    text.append(f'{inst} ### Review: {row["content"]} ### Reply: {row["replyContent"]}')
reviews['text'] = text
reviews = Dataset.from_pandas(reviews)
reviews = reviews.train_test_split(test_size=0.1)

In [17]:
reviews

DatasetDict({
    train: Dataset({
        features: ['replyContent', 'content', 'text', '__index_level_0__'],
        num_rows: 366
    })
    test: Dataset({
        features: ['replyContent', 'content', 'text', '__index_level_0__'],
        num_rows: 41
    })
})

In [18]:
reviews['train'][0]

{'replyContent': 'Hey JJ! Thank you for sharing your thoughts! We will share this with our team. Your feedback is what drives us to improve on our products and services, so do keep them coming! Have a great weekend ahead!💜🙏😊',
 'content': 'No option to add payees',
 'text': 'Below is a review for a banking app. Write a response that appropriately replies to the review from the perspective of the bank. ### Review: No option to add payees ### Reply: Hey JJ! Thank you for sharing your thoughts! We will share this with our team. Your feedback is what drives us to improve on our products and services, so do keep them coming! Have a great weekend ahead!💜🙏😊',
 '__index_level_0__': 130}

In [19]:
# baseline_model = "distilbert/distilgpt2"
baseline_model = "openai-community/gpt2" # not tested
tokenizer = AutoTokenizer.from_pretrained(baseline_model)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\AngJi\anaconda3\anaconda\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AngJi\.cache\huggingface\hub\models--openai-community--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [20]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [21]:
tokenized_reviews = reviews.map(preprocess_function, batched=True, remove_columns=reviews['train'].column_names)

Map:   0%|          | 0/366 [00:00<?, ? examples/s]

Map:   0%|          | 0/41 [00:00<?, ? examples/s]

In [22]:
tokenized_reviews

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 366
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 41
    })
})

In [23]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [24]:
model = AutoModelForCausalLM.from_pretrained(baseline_model)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [25]:
save_path = "./gpt_model_causallm"

training_args = TrainingArguments(
    output_dir=save_path,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_reviews["train"],
    eval_dataset=tokenized_reviews["test"],
    data_collator=data_collator,
)

trainer.train()
model.save_pretrained(save_path)


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
finetuned_model = AutoModelForCausalLM.from_pretrained(save_path)
generator = pipeline('text-generation', finetuned_model, tokenizer=tokenizer)

In [ ]:
review = reviews[50]['content']
prompt = f'{inst} ### Review: {review} ### Reply:'

In [ ]:
# generate output method 1
generator(prompt, max_length=len(review)+50)[0]['generated_text'][145+len(review):]

In [ ]:
# generate output method 2
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = finetuned_model.generate(input_ids, max_new_tokens = 50)
predicted_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(predicted_text[145+len(review):])

In [ ]:
def cosine_similarity(pred, actl):
    count_vectorizer = CountVectorizer()
    vector_matrix = count_vectorizer.fit_transform([pred[145:], actl[145:]])
    cosine_similarity_matrix = cosine_similarity(vector_matrix)
    return cosine_similarity_matrix[0,1]

In [ ]:
# not tested
cosine_similarity_ls = []
for review in reviews['test']:
    prompt = f'{inst} ### Review: {review['content']} ### Reply:'
    pred = generator(prompt, max_length=len(review['content'])+50)[0]['generated_text'][145+len(review['content']):]
    actl = review['replyContent']
    cosine_similarity_ls.append(cosine_similarity(pred, actl))

print(f'Average cosine similarity: {np.mean(cosine_similarity_ls)}')